# Data Wrangling with Pandas

Much of this notebook is based off [Chris' previous work](https://github.com/fonnesbeck/statistical-analysis-python-tutorial/)

Imports

In [ ]:
import numpy as np
import pandas as pd

Plotting setup

In [ ]:
%matplotlib inline
from seaborn import set_style
set_style("darkgrid")
import matplotlib.pyplot as plt

Display options

In [ ]:
import warnings
pd.set_option("max_rows", 10)
np.set_printoptions(suppress=True)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter("ignore", SettingWithCopyWarning)

## Date/time types

Pandas provides conveniences for working with dates

### Create a Range of Dates

In [ ]:
dates = pd.date_range("1/1/2015", periods=75, freq="D")

In [ ]:
dates

In [ ]:
y = pd.Series(np.random.randn(75), index=dates)

In [ ]:
y

In [ ]:
y.reset_index().dtypes

#### String indexing

You can index a DatetimeIndex with strings

In [ ]:
y.ix["2015-01-01":"2015-01-15"]

DatetimeIndex supports partial string indexing

In [ ]:
y["2015-01"]

You can resample to a lower frequency, specifying how to aggregate

In [ ]:
y.resample("M", how="mean")

Or go to a higher frequency, optionally specifying how to fill in the 

In [ ]:
y.asfreq("H", method='ffill')

There are convenience methods to lag and lead time series

In [ ]:
y

In [ ]:
y.shift(1)

In [ ]:
y.shift(-1)

To preserve the original data, you must specify the `freq`

In [ ]:
y.shift(-1, freq='D')

### Timestamps vs Periods

* Pandas provides a few data types building on [`np.datetime64`](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html)
* Timestamps are a point in time
* A sequences of Timestamps is a DatetimeIndex as we saw above

In [ ]:
pd.Timestamp("2015-1-1")

Periods represent a span of time

In [ ]:
p = pd.Period("Jan-2015")
p

In [ ]:
p.freq

In [ ]:
p = pd.Period("2015Q1")
p

The quarterly frequency is defined by when the fiscal year ends

In [ ]:
p.asfreq('D', 'start')

In [ ]:
p.asfreq('D', 'end')

In [ ]:
pd.PeriodIndex(start="2010Q1", periods=4)

In [ ]:
dates = pd.period_range("2010Q1", "2014Q2", freq="Q")
dates

PeriodIndex also supports partial string indexing

In [ ]:
py = pd.Series(np.random.randn(len(dates)), index=dates)
py['2010']

PeriodIndex provides the same conveniences as DatetimeIndex

In [ ]:
py.shift(-1, freq='Q')

In [ ]:
py.resample('M')

### Date Offsets

You can manipulate `Period` objects using arithmetic

In [ ]:
pd.Period("2015Q1") + 2

In [ ]:
pd.Period("2015Q1") - 2

For Timestamp objects, you need to use an [`Offset`](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#dateoffset-objects)

In [ ]:
pd.Timestamp("2015-1-1") + 2

Offests can be added / subtracted to / from a datetime object to obtain a shifted date

In [ ]:
pd.Timestamp("2015-1-1") + pd.DateOffset(months=2, days=5)

Subclasses of DateOffset implement custom date increments

In [ ]:
pd.datetime.now() + pd.datetools.Day()

Offsets can be multiplied by an integer (positive or negative) so that the increment will be applied multiple times

In [ ]:
d = pd.Timestamp("1-1-2015")

In [ ]:
d.strftime("%A")

In [ ]:
d + pd.datetools.BDay() * 5

You can also use parameterized Offsets

In [ ]:
d + pd.datetools.BDay(5)

Offsets have a rollforward and rollback methods for moving a date forward or backward to the next or previous "offset date"

In [ ]:
offset = pd.datetools.MonthEnd()

In [ ]:
offset.rollforward(d)

### Timedeltas

* [Timedeltas](http://pandas.pydata.org/pandas-docs/dev/timedeltas.html) are differences in times, expressed in difference units, e.g. days, hours, minutes, seconds. 
* They can be both positive and negative.
* pandas `Timedelta` is compatible with that of Python's `datetime`

You can construct a Timedelta object in many ways

From a string

In [ ]:
pd.Timedelta("2 days")

From an integer with a unit

In [ ]:
pd.Timedelta(2, "d")

From a DateOffset

In [ ]:
pd.Timedelta(pd.datetools.Second(5))

From a `datetime.timedelta`

In [ ]:
from datetime import timedelta
pd.Timedelta(timedelta(days=2, seconds=5))

In [ ]:
y = pd.date_range("1-1-2014", "1-31-2014", freq="D") + pd.datetools.Hour(12)

In [ ]:
y

In [ ]:
y - pd.Timedelta("12 hours 1 minute")

### Plotting

Pandas has time-series aware plotting capabilities. E.g., outside of the the Notebook, pan and zoom update the tick labels in a smart way.

In [ ]:
y = pd.Series(np.random.randn(50).cumsum(), 
              index=pd.date_range("1-1-2014", periods=50, freq="D"))

In [ ]:
y.plot(figsize=(10, 6))

### Rolling and Window Functions

Pandas also provides a number of convenience functions for working on rolling or moving windows of time series through a common interface

    pd.rolling_<TAB>

In [ ]:
pd.rolling_mean()

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', 
                                                          periods=1000))
ts = ts.cumsum()

rolling_mean = pd.rolling_mean(ts, window=60)

In [ ]:
ax = ts.plot(style='r--', figsize=(10, 10))
rolling_mean.plot(style='b', ax=ax)

**Exercise**

Use `pandas.io.data.DataReader` to download daily stock information for `GOOG`. Re-sample the closing price to the monthly average and compute the centered rolling yearly mean. Plot it vs. the monthly average as we did above. **NOTE:** `DataReader` is deprecated and the [pandas-datareader](https://github.com/pydata/pandas-datareader) package should be used in the future.

In [ ]:
from pandas.io import data

In [ ]:
y = data.DataReader(name="GOOG", data_source="google")

### AIS Data

In this section, we will manipulate data collected from ocean-going vessels on the eastern seaboard. Vessel operations are monitored using the Automatic Identification System (AIS), a safety at sea navigation technology which vessels are required to maintain and that uses transponders to transmit very high frequency (VHF) radio signals containing static information including ship name, call sign, and country of origin, as well as dynamic information unique to a particular voyage such as vessel location, heading, and speed.

The International Maritime Organization’s (IMO) International Convention for the Safety of Life at Sea requires functioning AIS capabilities on all vessels 300 gross tons or greater and the US Coast Guard requires AIS on nearly all vessels sailing in U.S. waters. The Coast Guard has established a national network of AIS receivers that provides coverage of nearly all U.S. waters. AIS signals are transmitted several times each minute and the network is capable of handling thousands of reports per minute and updates as often as every two seconds. Therefore, a typical voyage in our study might include the transmission of hundreds or thousands of AIS encoded signals. This provides a rich source of spatial data that includes both spatial and temporal information.

For our purposes, we will use summarized data that describes the transit of a given vessel through a particular administrative area. The data includes the start and end time of the transit segment, as well as information about the speed of the vessel, how far it travelled, etc.

In [ ]:
transit = pd.read_csv("../data/AIS/transit_segments.csv")

In [ ]:
transit.head()

Notice that the dates specified as an `object` dtype

In [ ]:
transit.info()

### Converting to Dates

First, let's convert the dates to something we can work with, using `to_datetime`

In [ ]:
pd.to_datetime(transit.st_time.head())

Or if you prefer day first...

In [ ]:
pd.to_datetime(transit.st_time.head(), dayfirst=True)

Pandas handles missing dates with a special `NaT` type

In [ ]:
pd.to_datetime([None])

Go ahead and convert them all

In [ ]:
transit["st_time"] = pd.to_datetime(transit.st_time)
transit["end_time"] = pd.to_datetime(transit.end_time)

In [ ]:
transit.head()

In [ ]:
transit.info()

Datetime types have a special `dt` accessor

In [ ]:
transit.st_time.dt.month

In [ ]:
transit.st_time.dt.dayofweek

## Merging and joining DataFrame objects

Let's have a peak at the transit data

In [ ]:
transit.seg_length.plot(kind="hist", bins=500)

* A few long trips make this difficult to read
* Let's look at a transformation of the data

In [ ]:
np.log(transit.seg_length).plot(kind="hist", bins=500)

Now that we have the vessel transit information as we need it, we may want a little more information regarding the vessels themselves. In the data/AIS folder there is a second table that contains information about each of the ships that traveled the segments in the segments table.

In [ ]:
vessels = pd.read_csv("../data/AIS/vessel_information.csv")

In [ ]:
vessels.head()

In [ ]:
vessels.info()

Look at each ship type (that doesn't contain `/`)

In [ ]:
vessels.ix[~vessels.type.str.contains("/"), "type"].unique()

In [ ]:
vessels.type.value_counts()

* Several ships in the vessels data have traveled multiple segments as we would expect
* Matching the names in the transit data to the vessels data is thus a many-to-one match
* We can combine these two datasets using the `merge` method
* Let's look at a trivial example of a merge

**Generator aside**

Generators in Python allow you to create a function that behaves like an iterator. I.e., a for loop. They are memory efficient since you do not need to keep the whole object in memory. Most things are generators by default in Python 3.

In [ ]:
def func_range(n):
    """
    This function returns a generator that counts from 0 to `n'
    """
    i = 0
    while i < n:
        yield i
        i += 1

In [ ]:
counter = func_range(10)

In [ ]:
counter

In [ ]:
for i in counter:
    print(i)

We've now consumed this generator

In [ ]:
for i in counter:
    print(i)

If we want a list, we have to ask for it.

In [ ]:
counter = list(func_range(10))
counter

Back to the merge example

In [ ]:
idx = list(range(3)) + list(range(3))
idx

In [ ]:
df1 = pd.DataFrame(dict(id=range(4), 
                        age=np.random.randint(18, 31, size=4)))

df2 = pd.DataFrame(dict(id=idx, 
                        score=np.random.random(size=6)))

In [ ]:
df1

In [ ]:
df2

By default, pandas identifies like-named columns to merge on

In [ ]:
df1.merge(df2)

* Also notice that id = 3 was dropped from the result
* By default, pandas does an **inner join** on a merge
  * This is the set intersection of the key(s)
* We can change this using the `how` keyword and specifying "other" for an **outer join**
  * This is the set union of the key(s)

In [ ]:
df1.merge(df2, how="outer")

We can use set operations to identify common keys beforehand

In [ ]:
vessels.columns.intersection(transit.columns)

We can see that this is a many-to-one merge

In [ ]:
vessels.shape

In [ ]:
transit.shape

In [ ]:
transit.merge(vessels).head()

You can also explicitly specify one or more keys to merge on

In [ ]:
dta = transit.merge(vessels, on="mmsi")

In [ ]:
dta.shape

In [ ]:
dta.head()

Have a look at the `merge`

In [ ]:
pd.merge?

**Watch out**, when merging on columns, indices are discarded

In [ ]:
A = pd.DataFrame(np.random.randn(25, 2), 
                 index=pd.date_range('1/1/2015', periods=25))
A[2] = np.repeat(list('abcde'), 5)

In [ ]:
A

In [ ]:
B = pd.DataFrame(np.random.randn(5, 2))
B[2] = list('abcde')

In [ ]:
B

In [ ]:
A.merge(B, on=2)

### Joins

* Join is like merge, but it works on the indices
* The same could be achieved with merge and the `left_index` and `right_index` keywords

In [ ]:
transit.set_index('mmsi', inplace=True)

In [ ]:
transit.join(vessels.set_index('mmsi')).head()

### Concatenation

* Another common operation is appending data row-wise or column-wise to an existing dataset
* We can use the `concat` function for this
* Let's import two microbiome datasets, each consisting of counts of microorganisms from a particular patient. 
* We will use the first column of each dataset as the index.
* The index is the unique biological classification of each organism, beginning with domain, phylum, class, and for some organisms, going all the way down to the genus level.

In [ ]:
mb1 = pd.read_excel('../data/microbiome/MID1.xls', index_col=0, 
                    header=None, names=["Taxon", "Counts"])
mb2 = pd.read_excel('../data/microbiome/MID2.xls', index_col=0, 
                    header=None, names=["Taxon", "Counts"])

In [ ]:
mb1.shape

In [ ]:
mb2.shape

In [ ]:
mb1.head()

In [ ]:
mb1.index.is_unique

In [ ]:
mb2.index.is_unique

We can concatenate on the rows

In [ ]:
pd.concat((mb1, mb2), axis=0).shape

Or concatenate column-wise

In [ ]:
pd.concat((mb1, mb2), axis=1)

Take the inner join while concatenating

In [ ]:
pd.concat((mb1, mb2), axis=1, join='inner')

Do an outer join and select the values from the first if present in both

In [ ]:
mb1.combine_first(mb2)

We can also create a hierarchical index based on keys identifying the original tables.

In [ ]:
pd.concat([mb1, mb2], keys=['patient1', 'patient2']).head()

**Exercise**

In the data/microbiome subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10th file that describes the content of each. Write code that imports each of the data spreadsheets and combines them into a single DataFrame, adding the identifying information from the metadata spreadsheet as columns in the combined DataFrame.

## FEC Individual Contributions Data

The individual contributions file contains each contribution from an individual to a federal committee if the contribution was at least \$200. The upper limit in 2012 w as \$2500.

A 25% sub-sample of the data is provided for you in the `../data` directory, but if you want the full dataset to play with later, you can download the original data [here](http://www.fec.gov/disclosurep/pnational.do). Click 2012 and export contributor data.

[Column Information](ftp://ftp.fec.gov/FEC/Presidential_Map/2012/DATA_DICTIONARIES/CONTRIBUTOR_FORMAT.txt)

```
CMTE_ID             COMMITTEE ID                
CAND_ID             CANDIDATE ID                
CAND_NM             CANDIDATE NAME              
CONTBR_NM           CONTRIBUTOR NAME           
CONTBR_CITY         CONTRIBUTOR CITY            
CONTBR_ST           CONTRIBUTOR STATE           
CONTBR_ZIP          CONTRIBUTOR ZIP CODE            
CONTBR_EMPLOYER     CONTRIBUTOR EMPLOYER            
CONTBR_OCCUPATION   CONTRIBUTOR OCCUPATION          
CONTB_RECEIPT_AMT   CONTRIBUTION RECEIPT AMOUNT     
CONTB_RECEIPT_DT    CONTRIBUTION RECEIPT DATE          
RECEIPT_DESC        RECEIPT DESCRIPTION         
MEMO_CD             MEMO CODE               
MEMO_TEXT           MEMO TEXT               
FORM_TP             FORM TYPE               
FILE_NUM            FILE NUMBER             
TRAN_ID             TRANSACTION ID              
ELECTION_TP         ELECTION TYPE/PRIMARY GENERAL INDICATOR 
```

In [ ]:
fec = pd.read_csv("../data/fec_subsample.csv.gz", compression="gzip")

In [ ]:
fec.columns

Let's rename a few columns

In [ ]:
fec.rename(columns=dict(contbr_city='city',
                        contbr_st='state',
                        contbr_zip='zip',
                        contbr_employer='employer',
                        contb_receipt_amt='amount',
                        contb_receipt_dt='date',
                        contbr_occupation='occupation',
                        contbr_nm='name',
                        cand_nm='candidate'
                        ), inplace=True)

In [ ]:
fec.info()

In [ ]:
fec.head()

In [ ]:
fec['date'] = pd.to_datetime(fec.date)

### Text data operations

Text fields have a `str` accessor that provide vectorized string operations

In [ ]:
fec.election_tp.unique()

We can do some data cleaning using `regex`. Regular expressions are used to do pattern matching on strings.

In [ ]:
import re

In [ ]:
re.search("\w+@[\w.]+", "Email: jon@thewall.com; Birthday: 1-1-1970").group()

In [ ]:
re.search("\d{1,2}[-/]\d{1,2}[-/]\d{4}", "Email: jon@thewall.com; Birthday: 1-1-1970").group()

It's common to use these operations to provide boolean indices

We use `na=False` to denote that NaNs should not be preserved but turned into False matches for boolean indexing to work.

In [ ]:
fec.election_tp.str.match("P$", na=False)

What primary elections do we have in the dataset?

In [ ]:
fec.ix[fec.election_tp.str.match("P$", na=False)].date.dt.year

Are they all in 2012?

In [ ]:
idx = fec.election_tp.str.match("P$", na=False)
fec.ix[idx].date.dt.year.isin([2012]).all()

Correct the typo we noticed above using the `replace` method

In [ ]:
fec['election_tp'] = fec.election_tp.str.replace('P2102', 'P2012')

Let's restrict our attention to the 2012 general election

In [ ]:
fec = fec.ix[fec.election_tp.str.match('G2012', na=False)]

In [ ]:
fec.shape

In [ ]:
fec.head()

If your index isn't meaningful, optionally you may reset it after re-indexing

In [ ]:
fec.reset_index(drop=True, inplace=True)

Replace method (not just for string data)

In [ ]:
to_replace = {'33': 'FL', 
              '46': 'IN',
              '48': 'MI',
              '49': 'MI',
              '7': 'NJ',
              '77': 'LA',
              '8': 'NJ',
              '84': 'UT',
              '91': 'CA'}

fec.replace(to_replace=dict(state=to_replace), inplace=True)

### Group By

* **Splitting** the data into groups
* **Applying** a function to each group separately
* **Combining** the results into a new data structure

#### Splitting step

* Abstractly, we use a mapping from labels to group names
* Mapping can be
  * Python function
  * A conformable list of group numbers
  * A dict mapping labels to groups
  * A column name
  * A list of any of the above


Grouping on a column

In [ ]:
fec.groupby("state")

Grouping on multiple columnns

In [ ]:
fec.groupby(['city', 'state'])

Grouping with a function

In [ ]:
def group_by_month(index):
    return index.month

We get back a `GroupBy` object

In [ ]:
grouper = fec.set_index('date').groupby(group_by_month)
grouper

The `groups` attribute is a dictionary mapping the unique groups to the axis labels that define them

In [ ]:
type(grouper.groups)

In [ ]:
grouper.groups.keys()

GroupBy with MultiIndex using one of the levels

In [ ]:
fec.set_index(['state', 'city']).head()

In [ ]:
fec.set_index(['state', 'city']).groupby(level='state').groups.keys()

You can do attribute lookup column selection on GroupBy objects

In [ ]:
grouper.amount

You can iterate through GroupBy objects much like through a dictionary's items

In [ ]:
for idx, group in grouper:
    print(idx)
    # print(group.head())
    # print()

In [ ]:
group.head()

Selecting a single group from a GroupBy object

In [ ]:
june = grouper.get_group(6)
print(june.amount.sum())

### Apply step

* A number of operations can be performed as part of the apply step
* **Aggregation**
  * E.g., group sums or means, sizes and counts
* **Transformation**
  * Changes that preserve the index. E.g., standardizing data or filling NaNs by group.
* **Filtration**
  * Discarding groups based on some criteria

#### Aggregation

In [ ]:
with pd.option_context("max_rows", 12):
    print(grouper.amount.aggregate(np.sum))

List the size of each group

In [ ]:
grouper.size()

Get summary statistics

In [ ]:
grouper.amount.describe()

* Other aggregation functions include

    `mean, sum, size, count, std, var, sem, describe, first, last, nth, min,` and `max`
   
* These functions are all `nan`-aware and will use the `bottleneck` library if installed
  * Bottleneck is used extensively in pandas such as in the statistical functions
  * It is a set of specialized cython routines that are especially fast when dealing with arrays that have nans

In [ ]:
grouper.mean()

* Notice that it automatically dropped the non-numeric columns, not exactly what we want with `file_num`

* You can use the `add_suffix` method to give the resulting column names that reflect the transformation
* Use `to_frame` to put the result back to a DataFrame

In [ ]:
grouper.amount.mean().to_frame().add_suffix("_mean")

* Alternatively, use a list to get a DataFrame in the first place

In [ ]:
grouper[['amount']].mean().add_suffix("_mean")

Passing custom or multiple functions

In [ ]:
grouper.amount.aggregate([np.mean, np.std, len])

Naming the output

In [ ]:
grouper.amount.aggregate({'average': np.mean, 'count': len})

Different functions to different columns

In [ ]:
def most_frequent(X):
    return X.to_frame().groupby(X.name).size().idxmax()

In [ ]:
with pd.option_context("max_rows", 12):
    print(grouper.aggregate(dict(state=most_frequent,
                                 amount=np.max)))

### Transformation

Transformations preserve the original index

In [ ]:
try:
    from scipy.stats import zscore
except:
    def zscore(x):
        return (x - x.mean())/x.std(ddof=0)

**Watch out** when working with pandas vs. numpy, pandas adjusts the degrees of freedom to give unbiased estimates whereas numpy does not

In [ ]:
np.std([1, 2, 3, 4])

In [ ]:
pd.Series([1, 2, 3, 4]).std()

In [ ]:
transformed = grouper.amount.transform(zscore)

In [ ]:
transformed.groupby(group_by_month).mean()

In [ ]:
transformed.groupby(group_by_month).std(ddof=0)

### Filtration

* The `filter` method allows us to return a subset of the original groups
* It takes a function that when applied to a group, return True or False
* Suppose we want to look only at months where the average donation is above the overall (positive donation) median

In [ ]:
median = fec.ix[fec.amount > 0].amount.median()

In [ ]:
fec.amount.median()

In [ ]:
fec.amount.mean()

In [ ]:
median

In [ ]:
grouper.filter(lambda x : x.ix[x.amount > 0].amount.mean() > median)

Let's filter to the 50 states plus DC using `filter`. We'll see another way to do this below.

In [ ]:
state_codes = {'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 
               'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 
               'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 
               'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 
               'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 
               'WY'}

In [ ]:
fec_states = fec.groupby("state").filter(lambda x: x.state.iloc[0] in state_codes)
set(fec_states.state.unique()).difference(state_codes)

In [ ]:
del fec_states

### Apply

Sometimes you need something different than aggregate and transform can provide. You can use apply for these cases.

In [ ]:
def func(group):
    return group.sum()

In [ ]:
grouper.amount.apply(func)

** Exercises **

Pick one of the following to do.

* Above we grouped by months, ignoring the year. Use a `groupby` to group by month and year to see the total amount contributed *for each candidate*

* This data contains refund information. Using the same GroupBy object you create above, how many refunds are there per month by party? (Barack Obama was the Democratic Party candidate. Jill Stein was the Green Party candidate. Gary Earl Johnson was the Libertarian Party candidate. Everyone else was a Republican.)

* What are the top 5 occupations by amount contributed to each candidate by candidate. Look at both by amount and number of contributions.

### Reshaping DataFrame objects

### Pivoting

The pivot method allows a DataFrame to be transformed easily between long and wide formats in the same way as a pivot table is created in a spreadsheet. 

It takes three arguments: index, columns and values, corresponding to the DataFrame index (the row headers), columns and cell values, respectively.

Let's look at the contributions by candidate across states over time

In [ ]:
fec_amounts = fec[["state", "date", "candidate", "amount"]]

This is in stacked format.

In [ ]:
fec_amounts.head()

Suppose we want to look at time series of the contributions to each candidate aggregated over the days

In [ ]:
fec_amounts = fec_amounts.groupby(["state", "date", "candidate"], 
                                  as_index=False).sum()

We can aggregate over the states to look at totals

In [ ]:
fec_totals = fec_amounts.groupby(["date", "candidate"], 
                                 as_index=False).amount.sum()

In [ ]:
fec_totals.head()

Pivot tables allow us to reshape the data so that we can look at the candidates as columns and the dates as the unique index

In [ ]:
fec_totals = fec_totals.pivot(index='date', 
                              columns='candidate', 
                              values='amount')
fec_totals.head()

Resample to monthly net donation amounts 

In [ ]:
fec_totals = fec_totals.resample("M", how='sum')

Make sure it's sorted

In [ ]:
fec_totals = fec_totals.sort_index()

In [ ]:
with pd.option_context("max_rows", 20):
    print(fec_totals.sum().sort(inplace=False))

In [ ]:
fec_totals.sum().idxmax()

In [ ]:
fec_totals.sum().idxmin()

Get the top candidates by fundraising

In [ ]:
candidates = fec_totals.sum().sort(inplace=False).index[7:]

In [ ]:
fec_totals[candidates].cumsum()

In [ ]:
ax = fec_totals[candidates].cumsum().plot(figsize=(10, 10))
ax.axvline(x=pd.Timestamp('6-23-2012'))

**Exercise**

Put the data in wide format with (candidate, state) MultiIndex and sorted monthly totals as columns

                         1/2012 2/2012 3/2012
    (Candidate, State)
    
**Hint**: Have a look at `DataFrame.pivot_table`   

### (Un)stacking data

* First put the data into a multi-index

In [ ]:
fec_monthly = fec_amounts.groupby(["state", "date", "candidate"]).sum()

In [ ]:
fec_monthly.head()

* **Unstack** turn the specified levels of the MultiIndex into columns

In [ ]:
fec_monthly.unstack(level=['state', 'candidate']).head()

* Now we have a DatetimeIndex and can resample

In [ ]:
fec_monthly = fec_monthly.unstack(level=['state', 
                                         'candidate']).resample('M', how='sum')

In [ ]:
fec_monthly.head()

* **Stack** is the inverse of unstack
* We can use it to go from 'wide' formats to 'long' formats

In [ ]:
fec_monthly.stack(level=["state"])

* These functions are intelligent about handling missing data
* If we want to preserve the full indices, we use `dropna=False`

In [ ]:
fec_monthly.stack(level=['state'], dropna=False)

### melt

We can also go from wide to long format using the `melt` function

In [ ]:
dta = pd.DataFrame({'first' : ['John', 'Mary'],
                    'last' : ['Doe', 'Bo'],
                    'height' : [5.5, 6.0],
                    'weight' : [130, 150]})

In [ ]:
dta

The ID variables uniquely index the rows and the columns are stacked to a single column with an identifier column.

In [ ]:
pd.melt(dta, id_vars=["first", "last"], var_name="quantity")

### Eval using NumExpr

* Expression evaluation of Series and DataFrame
* To benefit from using `eval`, you *must* have [numexpr](https://github.com/pydata/numexpr) installed
* The larger the DataFrame and the larger the expression, the larger the speed-ups you will see
  * Typically bigger than the cache size of your CPU
* Typical speed-ups can be 2x-4x to as much as 20x

#### NumExpr

* Fast numerical expression evaluator for NumPy
* Speed up operations on arrays (like "3*a+4*b")
* Avoids temporary arrays and uses less memory than doing the same calculation in Python.
  * Smart use of the cache
  * Smart chunking
  * Multi-threaded to help with problems that are not memory-bounded (e.g. those using transcendental functions)

#### Eval


* Arithmetic operations except for the left shift (<<) and right shift (>>) operators, e.g., `df + 2 * pi / s ** 4 % 42 - the_golden_ratio`
* Comparison operations, including chained comparisons, e.g., 2 < df < df2
* Boolean operations, e.g., df < df2 and df3 < df4 or not df_bool
* list and tuple literals, e.g., [1, 2] or (1, 2)
* Attribute access, e.g., df.a
* Subscript expressions, e.g., df[0]
* Simple variable evaluation, e.g., pd.eval('df') (this is not very useful)


* This Python syntax is not allowed:

* Expressions
  * Function calls
  * is/is not operations
  * if expressions
  * lambda expressions
  * list/set/dict comprehensions
  * Literal dict and set expressions
  * yield expressions
  * Generator expressions
  * Boolean expressions consisting of only scalar values
* Statements
  * Neither simple nor compound statements are allowed. This includes things like for, while, and if.

In [ ]:
columns = ['x{}'.format(i) for i in range(1, 31)]
dta = pd.DataFrame(np.random.randn(10000, 30), columns=columns)

In [ ]:
expr = " + ".join(columns)
expr

In [ ]:
dta.eval(expr)

You can use `eval` to do one assignment is allowed

In [ ]:
dta.eval("y = " + expr)

In [ ]:
dta.y

To distinguish between variable names in the DataFrame and local variables, prepend an `@` to the local variable name

In [ ]:
x1 = -50

In [ ]:
dta.eval("@x1 + " + expr)

### Eval Parser

You can use the `python` and `pandas` parsers to express queries using a more intuitive syntax

In [ ]:
dta.eval("(x1 > 0) & (x2 > 0) & (x3 > 0)", parser="python")

The pandas parser is even more lax

In [ ]:
dta.eval("x1 > 0 & x2 > 0 & x3 > 0", parser="pandas")

In [ ]:
dta.eval("x1 > 0 and x2 > 0 and x3 > 0", parser="pandas")

In [ ]:
del dta

### Performance Expectations

Large DataFrames 1-10mm rows

In [ ]:
from IPython.display import Image
Image("./eval-perf.png")

"Small" DataFrames (15k-20k rows) are faster in Python

In [ ]:
Image("./eval-perf-small.png")

## Advanced Indexing and Data Selection

## sample

New in `0.16.1`, Series and DataFrames provide a `sample` method for selecting random samples

In [ ]:
# %run load_fec

Specify number of rows

In [ ]:
fec.sample(n=1000)

You can specify a percentage

In [ ]:
fec.sample(frac=.05)

You can set a seed using an integer or a `numpy.random.RandomState` object

In [ ]:
rng = np.random.RandomState(100)
fec.sample(frac=.1, random_state=rng)

You can do boostrap (sampling with replacement)

In [ ]:
fec_bs = fec.sample(frac=.1, replace=True, random_state=rng)

In [ ]:
len(fec_bs.index.unique()) == fec_bs.shape[0]

In [ ]:
del fec_bs

We can also pass sampling weights. Say we want a sample that has roughly the same amount of people from each state.

In [ ]:
fec = fec.dropna(subset=['state'])

In [ ]:
populations = fec.groupby("state").size()

Make populations conformable to `fec`

In [ ]:
populations

In [ ]:
populations = populations.ix[fec.state]

Get inverse population weights

In [ ]:
weights = 1/populations

In [ ]:
weighted_fec = fec.sample(frac=.1, weights=weights, replace=True)

In [ ]:
weighted_fec.groupby("state").size()

## more permutation and sampling

In addition to the `sample` method, we could use numpy to do permutation. 

Calling `np.random.permutation` with the number of observations produces a shuffled index of integers.

In [ ]:
np.random.permutation(5)

In [ ]:
idx = np.random.permutation(len(fec))
idx = idx[:30]

In [ ]:
idx

In [ ]:
fec.take(idx)

Alternatively, we could shuffle the index inplace using `np.random.shuffle`. Recall, however, that indices are immutable, so we use an array representation accessed through `values`

In [ ]:
idx = fec.index.copy().values

In [ ]:
idx

In [ ]:
np.random.shuffle(idx)

In [ ]:
idx

In [ ]:
fec.ix[idx]

More on sampling methods later.

## scalar selection

If you only need one value then use `at` for label based lookup and `iat` for integers

In [ ]:
fec.columns[5]

In [ ]:
fec.iat[10, 5]

In [ ]:
fec.index[10]

In [ ]:
fec.at[27673, 'city']

## isin

We saw one way to filter using `groupby` above. Using `isin` is another way. It allows you to find columns that have a value contained in a set in a performant way.

In [ ]:
state_codes = {'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 
               'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 
               'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 
               'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 
               'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 
               'WY'}

In [ ]:
fec.state.isin(state_codes)

## where

* Recall that boolean indexing returns a subset of the data. 
* To keep the same shape as the original data, use the `where` method 
* This means that where must be a condition on the entire DataFrame or Series

In [ ]:
y = fec.amount
fec.where(y == 2500)

In [ ]:
y.where(y == 2500)

In [ ]:
dta = pd.DataFrame(np.random.randn(1000, 10))

In [ ]:
dta.where(dta < 0)

## mask

`mask` is the inverse of where. It masks the data where the condition is met

In [ ]:
dta.mask(dta < 0)

## query

The query method allows subset selection via an expression.

In [ ]:
fec.query("amount == 2500 | amount == -2500")

The query method is smart about using indices

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
df = pd.DataFrame(rng.randint(100 / 2, size=(100, 2)), 
                  columns=list('bc'))

In [ ]:
df.head()

In [ ]:
df.query("index > 10 and b < c")

If using a named index query falls back on the index if a column name is not found. This also works for a MultiIndex.

In [ ]:
df.index.name = 'a'

In [ ]:
df.query("a > 10 and b < c")

We can also use `in`/`notin` instead of the `isin` method

In [ ]:
df.query("a in [2, 3]")

This way of expression is powerfully succinct

In [ ]:
df.query("a not in b and b < c")

Compare to

In [ ]:
df.ix[~df.index.isin(df.b) & (df.b < df.c)]

We can compare DataFrames with the `equals` method.

In [ ]:
df.ix[~df.index.isin(df.b) & (df.b < df.c)].equals(df.query("a not in b and b < c"))

#### Exercise

The file `../data/weather_nyc.csv` contains some data on temperatures in New York City from 1995 to 2015. Let's clean up this data to get it into a format we will use later for plotting.

1. Load the data
2. Create 3 DataFrames
   1. Create a DataFrame called `past_stats` with a (month, day) index that contains for every year < 2014
      1. The daily low ("lower")
      2. The daily high ("upper")
      3. The average temperature ("avg")
      4. The standard error of the temperature ("std_err") (See below)
      5. Upper confidence interval for the average ("ci_upper") (See below)
      6. Lower confidence interval for the average ("ci_lower") (See below)
   2. Create a Series called `present_lows` with (month, day) index that contains *only* those days in 2014 for which the observed temperature was lower than the past lows.
      1. The values should be the observed low temperature.
   3. Create a Series called `present_highs` with (month, day) index that contains *only* those days in 2014 for which the observed temperature was higher than the past high.
   
The standard error of the temperature is given

$$\epsilon_x=\frac{\sigma_x}{\sqrt(n)}$$

where $\sigma_x$ is the standard deviation of `x`. The confidence interals are

$$(\mu_x - 1.96 \epsilon_x,\mbox{ } \mu_x + 1.96 \epsilon_x)$$

where $\mu_x$ is the average temperature.


## Odds and Ends

* Generating data

In [ ]:
pd.util.testing.makeDataFrame()

In [ ]:
pd.util.testing.makeMissingDataframe()

In [ ]:
pd.util.testing.makeTimeSeries()

* If you need to write unit tests

In [ ]:
df = pd.util.testing.makeDataFrame()
df2 = df.copy()

In [ ]:
pd.util.testing.assert_frame_equal(df, df2)

In [ ]:
df2 = pd.util.testing.makeDataFrame()

In [ ]:
pd.util.testing.assert_frame_equal(df, df2)

* There are also facilities for doing proper floating point comparisons

In [ ]:
df = df2.copy()

In [ ]:
df2.values[:, :] += 1e-6

In [ ]:
pd.util.testing.assert_frame_equal(df, df2)

In [ ]:
pd.util.testing.assert_frame_equal(df, df2, check_less_precise=True)